In [1]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import tqdm

from joblib import Parallel, delayed

In [2]:
with open('all.tsv') as f:
    object_names = [{
        'name': line[:-1].split('\t')[0],
        'glon': np.float64(line[:-1].split('\t')[1]),
        'glat': np.float64(line[:-1].split('\t')[2])
    } for line in f]
object_names[:3]

[{'name': 'PSZ2 G000.04+45.13', 'glon': 0.0405432, 'glat': 45.135175},
 {'name': 'PSZ2 G000.13+78.04', 'glon': 0.1380577, 'glat': 78.0421138},
 {'name': 'PSZ2 G000.40-41.86', 'glon': 0.4029953, 'glat': -41.8607926}]

In [3]:
size = (192, 192)

In [4]:
ROOT = '../data/resaved_HPXcvt'

def perform_object(glon, glat):
    hdu_100 = fits.open(f'{ROOT}/Planck100.fits')[0]
    wcs_100 = WCS(hdu_100.header)
    hdu_143 = fits.open(f'{ROOT}/Planck143.fits')[0]
    wcs_143 = WCS(hdu_143.header)
    hdu_217 = fits.open(f'{ROOT}/Planck217.fits')[0]
    wcs_217 = WCS(hdu_217.header)
    hdu_353 = fits.open(f'{ROOT}/Planck353.fits')[0]
    wcs_353 = WCS(hdu_353.header)
    hdu_545 = fits.open(f'{ROOT}/Planck545.fits')[0]
    wcs_545 = WCS(hdu_545.header)

    datasets = [
        (hdu_100, wcs_100),
        (hdu_143, wcs_143),
        (hdu_217, wcs_217),
        (hdu_353, wcs_353),
        (hdu_545, wcs_545)
    ]

    obj = np.zeros((5, size[0], size[1]))
    for freq_idx, (hdu, wcs) in enumerate(datasets):
        position = SkyCoord(glon, glat, frame='galactic', unit='deg')
        cutout = Cutout2D(hdu.data, position, size, wcs=wcs)
        data = np.array(cutout.data)
        if data.shape != size or np.isnan(data).any():
            return None
        obj[freq_idx] = data.astype(np.float32)
    return obj

results = []
while len(results) < 1564:# * 5:
    glon, glat = np.random.uniform(0, 360), np.random.uniform(-90, 90)
    obj = None
#     try:
    obj = perform_object(glon, glat)
#     except Exception as e:
#         print(e)
    if obj is not None and obj.shape == (5, size[0], size[1]):
        results.append((obj, None))
        if len(results) % 100 == 0:
            print(len(results))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500


In [5]:
isnone = 0
for i in results:
    if i is None:
        isnone += 1
print(isnone)

0


In [6]:
sz_name = np.array([x[1] for x in results if x is not None])
sz_data = np.stack([x[0] for x in results if x is not None])

In [7]:
np.savez_compressed('wo_sz.npz', sz_data=sz_data, sz_names=sz_name)

In [8]:
sz_name.shape

(1564,)